In [3]:
import sqlalchemy as sa
import urllib 
import pandas as pd 
import psycopg2
from sqlalchemy import text
from conector import enginok

Leemos el .csv/museos para la creacion y normalizacion de la misma

In [4]:
dataM = pd.read_csv("./museos.csv", encoding='UTF-8')
df_Museos = pd.DataFrame(dataM)
df_Museos.sample(2)

,Cod_Loc,IdProvincia,IdDepartamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,direccion,...,Mail,Web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,aÃ±o_inauguracion,actualizacion
702,46112010,46,46112,NaN,Espacios de ExhibiciÃ³n Patrimonial,Museos,La Rioja,Villa Chepes,Museo De Los Caudillos Ãngel Nardillo,"9 de Julio, Esquina Sarmiento",...,NaN,NaN,-31.341160,-66.592516,LocalizaciÃ³n precisa,NaN,DNPyM,Municipal,NaN,2017
726,50070060,50,50070,NaN,Espacios de ExhibiciÃ³n Patrimonial,Museos,Mendoza,MaipÃº,Museo Municipal Kakel Huincul,Belgrano 349,...,culturamaipu@infovia.com.ar,NaN,-36.863451,-57.885064,LocalizaciÃ³n precisa,NaN,DNPyM,Municipal,2005.0,2017


CREAR UN ID UNICO UTILIZANDO EL INICE.

In [5]:
df_Museos['Id']=df_Museos.index
df_Museos['Id']=df_Museos['Id'].astype(str)
df_Museos['Id']=df_Museos['Id']+'M'
df_Museos.sample()

,Cod_Loc,IdProvincia,IdDepartamento,Observaciones,categoria,subcategoria,provincia,localidad,nombre,direccion,...,Web,Latitud,Longitud,TipoLatitudLongitud,Info_adicional,fuente,jurisdiccion,aÃ±o_inauguracion,actualizacion,Id
514,14014010,14,14014,NaN,Espacios de ExhibiciÃ³n Patrimonial,NaN,CÃ³rdoba,Cordoba,"Museo de ZoologÃ­a, Academia de ciencias",VÃ©lez SÃ¡rsfield 249,...,NaN,-31.418144,-64.187804,LocalizaciÃ³n precisa,NaN,RCC- CÃ³rdoba,Municipal,0.0,2017,514M


**RENOMBRAR Y ORDENAR COLUMBNAS**

In [6]:
df_Museos.rename({'aÃ±o_inauguracion':'Ano_inauguracion','actualizacion':'Ano_actualizacion',
'direccion':'Direccion','provincia':'Provincia', 'fuente':'Fuente', 'categoria':'Categoria','Observaciones':'Observacion','nombre':'Nombre',
'piso':'Piso','cod_area':'Cod_tel','telefono':'Telefono','jurisdiccion':'Jurisdiccion','localidad':'Localidad'},
axis=1, inplace=True)

In [7]:
df_Museos.drop(['Provincia','Localidad','IdDepartamento','CP', 'Cod_tel','IdProvincia',],axis=1,inplace=True)

In [8]:
df_Museos=df_Museos[['Id', 'Cod_Loc','Nombre','Categoria', 'subcategoria',
       'Direccion', 'Observacion','Piso',  'Web' , 'Info_adicional',
       'Telefono', 'Mail', 'Latitud', 'Longitud', 'TipoLatitudLongitud',
       'Fuente', 'Jurisdiccion', 'Ano_inauguracion', 'Ano_actualizacion']]    

**IMPORTACION DE TIPOS DE DATOS QUE UTILIZARE PARA CREAR LA TABLA SIGUIENTE**

In [9]:
from sqlalchemy.types import VARCHAR 
from sqlalchemy.types import INTEGER
from sqlalchemy.types import Text
from sqlalchemy.types import NUMERIC

**CRAER TABLA "Museos"**

In [10]:
df_Museos.to_sql('MUSEOS', con=enginok, if_exists='replace', index=False,
          dtype={"Id":VARCHAR,"Cod_Loc": INTEGER, "Categoria":Text,"subcategoria":Text,
          "Observacion":Text, "Nombre":Text, "Telefono":VARCHAR(200) , "Mail": VARCHAR(200),
          "Web":VARCHAR,  "Info_adicional":Text, "Fuente":VARCHAR(200), 'Latitud':NUMERIC, 'Longitud':NUMERIC, 'TipoLatitudLongitud':Text,
          "Jurisdiccion":VARCHAR(200) , "Ano_inauguracion":INTEGER, "Ano_actualizacion":INTEGER})

182

In [11]:
text=("""SELECT * FROM "MUSEOS"
    WHERE "Cod_Loc"=86119100""")


muestreo = pd.DataFrame(enginok.execute(text))
muestreo.head(2)


""


creando llave Primaria de la tabla Museos

In [12]:
text=("""ALTER TABLE "MUSEOS" ADD PRIMARY KEY ("Id");""") 
enginok.execute(text)

Relacines correspondientes 'FOREIGN KEY'

In [14]:
text=("""ALTER TABLE  "MUSEOS"
ADD FOREIGN KEY ( "Cod_Loc") REFERENCES "Localidad" ("Cod_Loc") ;""")
enginok.execute(text)
 